In [698]:
import pandas as pd
import re
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Charger l'extraction Tabula

In [699]:
df = pd.read_csv('tabula-compenswiss - list of positions - 30.06.2022.csv')

## Nettoyer l'extraction Tabula
J'ai gardé les entêtes de chaque page pour garder les bonnes colonnes lors de l'extraction. On a plus besoin de ces entêtes.

In [700]:
df = df[df['ISIN'] != 'ISIN']

Formate les nombres

In [701]:
# Supprime les séparateurs de milliers
df['Market Value CHF'] = df['Market Value CHF'].replace("'",'', regex=True)

# Supprime les décimales
df['Market Value CHF'] = df['Market Value CHF'].replace("\..*",'', regex=True)

# Convertit les montants en entiers
df['Market Value CHF'] = df['Market Value CHF'].astype(int)

In [702]:
df.head()

,ISIN,Asset Class,Description,Nominal/Quantity,Market Value CHF
0,XS2459025909,Money market Investments,0 ALLZ 22 ECP,EUR 3'000'000,3008851
1,CH0496692960,Money market Investments,0 BALOISE HLDG 22,CHF 8'100'000,8108100
2,CH0496692960,Money market Investments,0 BALOISE HLDG 22,CHF 8'400'000,8392440
3,CH0496692960,Money market Investments,0 BALOISE HLDG 22,CHF 1'000'000,999100
4,CH0407153342,Money market Investments,0 BLKB 23,CHF 5'000'000,4972500


## Vérifier les montants

In [703]:
df['Market Value CHF'].sum()

34454523303

In [704]:
len(df)

5514

## Mapping Bloomberg pour identifier les compagnies et leur catégorie d'activité


In [705]:
### Charger le fichier Bloomberg (Finanz & Wirtschaft)
### Créer un dictionnaire avec des ISIN uniques pour identifier l'entreprise

bloom = pd.read_excel('mapping.xlsx')

bloom = bloom.drop_duplicates(subset=['ISIN'])

In [706]:
df = pd.merge(bloom, df, on = "ISIN", how = "right", indicator = True)

# 234 lignes du fichier Compenswiss ne peuvent pas être identifiées
df['_merge'].value_counts()

both          5280
right_only     234
left_only        0
Name: _merge, dtype: int64

In [707]:
# Exemple

df[df['ISIN'] == 'CA01626P3043']

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
3824,CA01626P3043,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Equities,ALIMENT COUCHE -A,43'286,1613004,right_only


In [804]:
df[df['_merge'] == 'right_only'].head()

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
0,XS2459025909,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 ALLZ 22 ECP,EUR 3'000'000,3008851,right_only
8,XS2435086652,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 DNBB 22 ECP,EUR 3'000'000,3015160,right_only
19,XS2489589874,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 OPCB 23 ECP,EUR 3'500'000,3503016,right_only
20,XS2429878700,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 OPCB 23 ECP,USD 2'000'000,1903345,right_only
25,FR0127437160,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 SG 23 ECP,USD 1'000'000,957350,right_only


## Tests for manual update

In [ ]:
# Méthode pour combler les lacunes d'un dataframe avec un dataframe à la structure similaire

In [709]:
df1 = pd.DataFrame({'id':[1,2],
                    'company': ['Novartis', np.NaN],
                    'category': [np.NaN, "NaN"]
                   })
df2 = pd.DataFrame({'id': [1,2],
                    'company': [np.NaN, 'Migros'],
                    'category': ["pharma", "retail"]
                   })

df1 = df1.set_index("id").combine_first(df2.set_index("id")).reset_index()
df1

## Editing

In [665]:
# Charger le fichier contenant des ajouts manuels faits sur Google Sheets

transfo = pd.DataFrame(pd.read_csv('transformations_manuelles.csv'))

In [666]:
transfo.sample()

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Currency,Nominal/Quantity,Market Value CHF,Date.1,_merge
4,US42225P5017,NaN,NaN,NaN,NaN,NaN,HEALTHCARE TRUST AME. HTA.,NaN,NaN,Engineering&Construction (medical office build...,NaN,NaN,NaN,NaN,NaN,NaN,Real Estate,HLTHC RLTY TR-A,NaN,150246,4014519,False,right_only


In [667]:
# Fill df NaNs with transfo values (Company Name and Gruppe), 10 rows modified
# Combler les cellules manquants des colonnes Company Name et Gruppe (NaN) du fichier Compenswiss avec les ajouts manuels

df = df.set_index("ISIN").combine_first(transfo.set_index("ISIN")).reset_index()


In [668]:
# Exemples

df[df.apply(lambda row: row.astype(str).str.contains("US276480AE09", case=False).any(), axis=1)]


,ISIN,Asset Class,Company Name,Country (P),Country ISO,Currency,Date,Date.1,Description,Emittent,Gruppe,Kurzname,Land (Risiko),Market Value CHF,Name,Nominal/Quantity,Sektor,Sitzland,Ticker,Untergruppe,Vollständiger Name,Wertpapiertyp,_merge
3582,US276480AE09,Foreign Currency bonds,Eastern Gas Transmission & Storage Inc,NaN,NaN,NaN,NaT,False,4.8 EAST 43NTS-144A,NaN,Gas,NaN,NaN,559122.0,NaN,USD 625'000,NaN,NaN,NaN,NaN,NaN,NaN,right_only


In [669]:
df[df.apply(lambda row: row.astype(str).str.contains("XS2429878700", case=False).any(), axis=1)]


,ISIN,Asset Class,Company Name,Country (P),Country ISO,Currency,Date,Date.1,Description,Emittent,Gruppe,Kurzname,Land (Risiko),Market Value CHF,Name,Nominal/Quantity,Sektor,Sitzland,Ticker,Untergruppe,Vollständiger Name,Wertpapiertyp,_merge
5384,XS2429878700,Money market Investments,OP YRITYSPANK,NaN,NaN,NaN,NaT,True,0 OPCB 23 ECP,NaN,Banks,NaN,NaN,1903345.0,NaN,USD 2'000'000,NaN,NaN,NaN,NaN,NaN,NaN,right_only


## Analyses

#### La valeur la plus haute est un fond Pictet

In [672]:
df[df['Market Value CHF'] == df['Market Value CHF'].max()]

,ISIN,Asset Class,Company Name,Country (P),Country ISO,Currency,Date,Date.1,Description,Emittent,Gruppe,Kurzname,Land (Risiko),Market Value CHF,Name,Nominal/Quantity,Sektor,Sitzland,Ticker,Untergruppe,Vollständiger Name,Wertpapiertyp,_merge
456,CH0180951227,Equities,Pictet CH Institutional - Emerging Markets Tra...,0,0,NaN,2022-06-30,NaN,PICTCHEMMKTT Z USD,Pictet CH Institutional - Emer,Equity Fund,PIC-EM MK T-Z$,CH,1.787431e+09,PICTET CH-EMERGING MK T-ZUSD,1'122'632,Funds,CH,PICEMZU,Emerging Market-Equity,PICTET CH-EMERGING MK T-ZUSD,Open-End Funds,both


In [783]:
df.nlargest(10, 'Market Value CHF')

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
4810,CH0180951227,2022-06-30,PICTET CH-EMERGING MK T-ZUSD,PICEMZU,Open-End Funds,Pictet CH Institutional - Emer,Pictet CH Institutional - Emerging Markets Tra...,PICTET CH-EMERGING MK T-ZUSD,PIC-EM MK T-Z$,Equity Fund,Funds,Emerging Market-Equity,0,0,CH,CH,Equities,PICTCHEMMKTT Z USD,1'122'632,1787431257,both
5513,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commodities,GOLD - LBMA GOOD DELIVERY BARS,NaN,1183595274,right_only
333,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loans,COMPENSWISS LOAN PORTFOLIO,NaN,1141047419,right_only
3591,IE00BCQNQM04,2022-06-30,LEGG MAS US MORT BACK-A USD,WANBLAU,Open-End Funds,Legg Mason Western Asset US Mo,Legg Mason Western Asset US Mortgage-Backed Se...,LEGG MAS US MORT BACK-A USD,LEGG-US MOR-A$,Debt Fund,Funds,Unclassified,0,0,IE,IE,Foreign Currency bonds,LM WA MBS USD-LM-AC,10'358'850,1023083345,both
5367,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Real Estate,Immobilien Schweiz II,NaN,936352131,right_only
5366,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Real Estate,Immobilien Schweiz I,NaN,925025379,right_only
3397,IE00B6YX4R11,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Currency bonds,BAR EU LN EUR-TB-AC,4'109'825,654762153,right_only
4454,CH0244037112,2022-06-30,ISH W EX SW S/C EQ IN-X0NCHF,BSSXCX0,Open-End Funds,iShares World ex Switzerland S,iShares World ex Switzerland Small Cap Equity ...,ISH W EX SW S/C EQ IN-X0NCHF,ISH-EXW S/C-X0CH,Equity Fund,Funds,Unclassified,0,0,CH,CH,Equities,ISWSSCEIFC-X0 N CHF,364'603,618622486,both
3548,IE00B6ZNT534,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Currency bonds,GUGG U.S. LN USD-AC,3'678'439,562851237,right_only
3590,IE00BYQP5H80,2022-06-30,LEGG MAS WES AST STR OP-AUSD,WASOLAU,Open-End Funds,Legg Mason Western Asset Struc,Legg Mason Western Asset Structured Opportunit...,LEGG MAS WES AST STR OP-AUSD,LEGG-AST OPP-A$,Debt Fund,Funds,Unclassified,0,0,IE,IE,Foreign Currency bonds,LM STR OP USD-LM-AC,3'020'973,391766317,both


#### Comptabiliser les montants investis par groupe d'activité

#### Oil & Gas

In [805]:
oil = df[(df['Gruppe'] == 'Oil and gas') | 
         (df['Gruppe'] == 'Oil&Gas') | 
         (df['Gruppe'] == 'Oil&Gas Services') |
         (df['Gruppe'] == 'Gas') | 
         (df['Gruppe'] == 'Pipelines') 
        ]
        

oil[['Company Name', 'Gruppe', 'Market Value CHF']].head()

,Company Name,Gruppe,Market Value CHF
78,Korea National Oil Corp,Oil&Gas,10216908
97,EBN BV,Oil&Gas,1048005
268,BP Capital Markets PLC,Oil&Gas,477105
297,TotalEnergies Capital SA,Oil&Gas,2417321
307,ONEOK Partners LP,Pipelines,2678516


In [758]:
oil['Market Value CHF'].sum()

620079747

#### Tobacco

In [718]:
tobacco = df[df.apply(lambda row: row.astype(str).str.contains("tobacco", case=False).any(), axis=1)]

In [719]:
tobacco['Market Value CHF'].sum()

52515777

In [720]:
tobacco[['Company Name', 'Gruppe', 'Market Value CHF']]

,Company Name,Gruppe,Market Value CHF
1355,BAT International Finance PLC,Agriculture,1942000
1437,Philip Morris International Inc,Agriculture,4012000
2482,BAT International Finance PLC,Agriculture,1041276
3071,BAT International Finance PLC,Agriculture,2536086
3161,BAT Capital Corp,Agriculture,2362594
3169,BAT International Finance PLC,Agriculture,7406929
3207,BAT Capital Corp,Agriculture,4953195
3836,Altria Group Inc,Agriculture,3702016
3982,British American Tobacco PLC,Agriculture,9377954
4421,Imperial Brands PLC,Agriculture,2031572


#### Armes, militaire, défense

In [713]:
weapons = df[(df['Gruppe'] == 'Aerospace/Defense') | 
             (df['Untergruppe'] == 'Explosives')
            ]

len(weapons)


30

In [714]:
weapons['Market Value CHF'].sum()

98385932

#### Nucléaire

In [744]:
nuclear = df[df.apply(lambda row: row.astype(str).str.contains('nuclear', case=False).any(), axis=1)]
nuclear

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
360,CH0485445958,2022-06-30,KOHNPW 0 07/19/24,KOHNPW,Corporate Bonds,KOREA HYDRO & NUCLEAR PO,Korea Hydro & Nuclear Power Co Ltd,KOHNPW 0 07/19/24,KOREA HYDRO & NU,Electric,Utilities,Electric-Generation,S.Korea,KR,KR,KR,Swiss Franc bonds,0 KOR HYD 24 GMTN,CHF 1'000'000,974500,both
450,CH0485445966,2022-06-30,KOHNPW 0.05 07/19/27,KOHNPW,Corporate Bonds,KOREA HYDRO & NUCLEAR PO,Korea Hydro & Nuclear Power Co Ltd,KOHNPW 0.05 07/19/27,KOREA HYDRO & NU,Electric,Utilities,Electric-Generation,S.Korea,KR,KR,KR,Swiss Franc bonds,0.05 KOREAHYDRO 27,CHF 1'000'000,914000,both


In [745]:
nuclear['Market Value CHF'].sum()

1888500

#### Voitures

In [753]:
auto = df[(df['Gruppe'] == 'Auto Parts') | 
         (df['Gruppe'] == 'Auto ABS') | 
         (df['Gruppe'] == 'Auto Manufacturers') |
         (df['Gruppe'] == 'Auto Parts&Equipment') | 
         (df['Untergruppe'] == 'Retail-Automobile') | 
         (df['Untergruppe'] == 'Retail-Auto Parts') | 
         (df['Untergruppe'] == 'Rental Auto/Equipment')
        ]
auto['Market Value CHF'].sum()


367270480

#### Mining

In [682]:
mining = df[(df['Gruppe'] == 'Mining')| 
         (df['Untergruppe'] == 'Machinery-Constr&Mining')
           ]



mining[['Company Name', 'Gruppe', 'Untergruppe', 'Market Value CHF']].groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

In [752]:
mining['Market Value CHF'].sum()

104412840.0

In [751]:
mining['Company Name'].unique()

array(['BHP Group Ltd', 'Evolution Mining Ltd', 'Newcrest Mining Ltd',
       'Northern Star Resources Ltd', 'Rio Tinto Ltd', 'South32 Ltd',
       'Silfin NV', 'Agnico Eagle Mines Ltd', 'Barrick Gold Corp',
       'Cameco Corp', 'First Quantum Minerals Ltd', 'Franco-Nevada Corp',
       'Ivanhoe Mines Ltd', 'Kinross Gold Corp', 'Lundin Mining Corp',
       'Pan American Silver Corp', 'Teck Resources Ltd',
       'Wheaton Precious Metals Corp', 'Glencore Finance Europe Ltd',
       'Glencore Capital Finance DAC', 'Antofagasta PLC', 'Rio Tinto PLC',
       'Anglo American PLC', 'Glencore PLC', 'Komatsu Ltd',
       'Sumitomo Metal Mining Co Ltd',
       'Hitachi Construction Machinery Co Ltd', 'Norsk Hydro ASA',
       'Sandvik AB', 'Epiroc AB', 'Boliden AB', 'Alcoa Corp',
       'Barrick North America Finance LLC',
       'Barrick PD Australia Finance Pty Ltd', 'Caterpillar Inc',
       'Caterpillar Financial Services Corp', 'Freeport-McMoRan Inc',
       'Newmont Corp', 'BHP Billiton 

#### Total Energies

In [683]:
total = df[df.apply(lambda row: row.astype(str).str.contains('total', case=False).any(), axis=1)]

total['Company Name'].value_counts()

TotalEnergies SE                          6
TotalEnergies Capital International SA    3
TotalEnergies Capital SA                  1
Name: Company Name, dtype: int64

In [715]:
total['Market Value CHF'].sum()

57855780.0

#### Amazon

In [716]:
amazon = df[df.apply(lambda row: row.astype(str).str.contains('amazon', case=False).any(), axis=1)]

amazon['Company Name'].value_counts()

Amazon.com Inc    1
Name: Company Name, dtype: int64

In [717]:
amazon['Market Value CHF'].sum()

47539941

#### BHP

In [725]:
bhp = df[df.apply(lambda row: row.astype(str).str.contains('BHP', case=False).any(), axis=1)]

In [726]:
bhp['Market Value CHF'].sum()

20286132

#### Energies renouvelables

In [728]:
en = df[df['Gruppe'] == 'Energy-Alternate Sources']

en['Market Value CHF'].sum()

26552636

In [730]:
en[['Company Name', 'Gruppe', 'Market Value CHF']]

,Company Name,Gruppe,Market Value CHF
184,Grande Dixence SA,Energy-Alternate Sources,2000600
806,Grande Dixence SA,Energy-Alternate Sources,892500
1203,Grande Dixence SA,Energy-Alternate Sources,1917000
1362,Grande Dixence SA,Energy-Alternate Sources,446400
1761,EEW Energy from Waste GmbH,Energy-Alternate Sources,974889
1763,Acciona Energia Financiacion Filiales SA,Energy-Alternate Sources,1308076
1844,ERG SpA,Energy-Alternate Sources,5195072
3902,Atlantica Sustainable Infrastructure PLC,Energy-Alternate Sources,3636357
4219,Enphase Energy Inc,Energy-Alternate Sources,1275307
4277,First Solar Inc,Energy-Alternate Sources,2743789


#### Viande

In [734]:
viande = df[df['Untergruppe'] == 'Food-Meat Products']
viande[['Company Name', 'Gruppe', 'Market Value CHF']]

,Company Name,Gruppe,Market Value CHF
865,Bell Food Group AG,Food,1820400
1100,Bell Food Group AG,Food,2624400
1162,Bell Food Group AG,Food,926000
1413,Bell Food Group AG,Food,2384375
3003,Tyson Foods Inc,Food,3646297
3302,JBS USA LUX SA / JBS USA Food Co / JBS USA Fin...,Food,4535446
4393,Hormel Foods Corp,Food,692026
5129,Tyson Foods Inc,Food,1231476
5209,WH Group Ltd,Food,671707


In [733]:
viande['Market Value CHF'].sum()

18532127

#### Eau

In [737]:
df[df.apply(lambda row: row.astype(str).str.contains('water', case=False).any(), axis=1)]['Gruppe'].value_counts()

Water                    7
Banks                    7
Environmental Control    3
Healthcare-Products      1
Machinery-Diversified    1
Name: Gruppe, dtype: int64

In [736]:
eau = df[df['Gruppe'] == 'Water']


In [738]:
eau['Market Value CHF'].sum()

12519789

In [739]:
eau

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
601,CH0429659623,2022-06-30,KORWAT 0.15 09/20/23,KORWAT,Corporate Bonds,KOREA WATER RESOURCES,Korea Water Resources Corp,KORWAT 0.15 09/20/23,KOREA WATER RESO,Water,Utilities,Water,S.Korea,KR,KR,KR,Swiss Franc bonds,0.15 KOREAWATER 23,CHF 1'000'000,988900,both
2794,XS1627343186,2021-12-31,AQUASM 2.629 06/08/27,AQUASM,Corporate Bonds,FCC AQUALIA SA,FCC Aqualia SA,AQUASM 2.629 06/08/27,FCC AQUALIA SA,Water,Utilities,Water,Spain,ES,ES,ES,Foreign Currency bonds,2.629 FCC AQUALIA27,EUR 3'500'000,3239238,both
3843,US0304201033,2022-06-30,AMERICAN WATER WORKS CO INC,AWK,Common Stocks,American Water Works Co Inc,American Water Works Co Inc,AMERICAN WATER WORKS CO INC,AMERICAN WATER W,Water,Utilities,Water,United States,0,US,US,Equities,AMER WTR WORKS RG,9'329,1328682,both
4235,US29670G1022,2022-06-30,ESSENTIAL UTILITIES INC,WTRG,Common Stocks,Essential Utilities Inc,Essential Utilities Inc,ESSENTIAL UTILITIES INC,ESSENTIAL UTILIT,Water,Utilities,Water,United States,0,US,US,Equities,ESSENTIAL UTIL RG,12'480,547803,both
4942,GB00B1FH8J72,2022-06-30,SEVERN TRENT PLC,SVT,Common Stocks,Severn Trent PLC,Severn Trent PLC,SEVERN TRENT PLC,SEVERN TRENT,Water,Utilities,Water,United Kingdom,0,GB,GB,Equities,SEVERN TRENT RG,26'129,826004,both
5144,GB00B39J2M42,2022-06-30,UNITED UTILITIES GROUP PLC,UU/,Common Stocks,United Utilities Group PLC,United Utilities Group PLC,UNITED UTILITIES GROUP PLC,UNITED UTILITIES,Water,Utilities,Water,United Kingdom,0,GB,GB,Equities,UNITED UTILITIES RG,71'189,844235,both
5160,FR0000124141,2022-06-30,VEOLIA ENVIRONNEMENT,VIE,Common Stocks,Veolia Environnement SA,Veolia Environnement SA,VEOLIA ENVIRONNEMENT,VEOLIA ENVIRONNE,Water,Utilities,Water,France,0,FR,FR,Equities,VEOLIA ENVIRONNEM.,203'557,4744927,both


In [ ]:
eau['Market Value CHF'].sum()

#### Papier

In [742]:
papier = df[df['Gruppe'] == 'Forest Products&Paper']

In [743]:
papier['Market Value CHF'].sum()

14060044

#### Air travel


In [746]:
df[(df['Gruppe'] == 'Airlines') | (df['Untergruppe'] == 'Airport Develop/Maint')][['Company Name', 'Gruppe', 'Untergruppe', 'Market Value CHF']].groupby('Company Name')['Market Value CHF'].sum()

Company Name
ANA Holdings Inc                                        565397
Aena SME SA                                            2801347
Aeroport International de Geneve                       6396600
Aeroporti di Roma SpA                                   447205
Aeroports de Paris                                     1086168
Air Canada                                              104079
Auckland International Airport Ltd                      585635
Avinor AS                                               658647
Brussels Airport Co SA                                 2041758
DAA Finance PLC                                         822308
Delta Air Lines Inc                                     231194
Delta Air Lines Inc / SkyMiles IP Ltd                  2754475
Deutsche Lufthansa AG                                  1001592
Flughafen Zurich AG                                   28866040
Heathrow Funding Ltd                                  10744620
Japan Airlines Co Ltd                     

In [755]:
df[(df['Gruppe'] == 'Airlines') | (df['Untergruppe'] == 'Airport Develop/Maint')]['Market Value CHF'].sum()

87164456

In [757]:
df[(df['Gruppe'] == 'Airlines') | (df['Untergruppe'] == 'Airport Develop/Maint')].groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

#### Entreprises polémiques

In [785]:
target = [
"Amazon",
"TotalEnergies",
"Toyota",
"Engie",
"Air Liquide",
"Enel",
"Iberdrola",
"Shell",
"Exxon",
"Gulf Power",
"Vinci",
"Var Energi",
"Monongahela Power Company - Mon Power",
"Monongahela",
"Philip Morris PMI",
"PMI",
"Philip Morris",
"Airbus",
"Flughafen Zurich",
"ZRH",
"Safran",
"Dassault",
"SIKA",
"Holcim",
"Danone",
"Shin-Etsu Chemical",
"Shin-Etsu",
"Pepsi",
"Bayer",
"Pernod-Ricard",
"Diageo",
"Lonza",
"Henkel",
"Walmart",
"BASF",
"Nestlé",
"Coca Cola",
"Unilever",
"Procter and Gamble",
"Procter & Gamble",
"Procter&Gamble",
"Mercedes",
"Tesla",
"Mitsubishi",
"Schneider Electric",
"ABB",
"LVMH",
"Siemens",
"Stellantis",
"Huntington Ingalls Industries Inc",
"Huntington Ingalls",
"BHP Ltd",
"EDF",
"Arcelor Mittal",
"Air Canada",
"Continental Airlines",
"Cimic group",
"Ballard Systems Power",
"Atco ltd/Canada",
"Ivanhoe Mines Ltd",
"Pan American Silver Corp",
"Lundin Mining Corp",
"Fresnillo PLC",
"Magellan Midstream Partner",
"Parkland Corp",
"Yamana Gold inc",
"Kinross Gold",
"Keyera corp",
"Delta Airlines",
"B2 Gold corp",
"AltaGas LTD",
"Heico Corp",
"SouthWest Airlines",
"Mercury NZ",
"Qantas Airways",
"Qantas",
"Diamondback Energy inc",
"Evolution mining ltd",
"Air China",
"Coal india",
"Indian oil",
"Petronas",
"Polish oil and gas company",
"Ugi corp",
"Imperial oil ltd"]

In [787]:
#df_target = df[df.apply(lambda row: row.astype(str).str.contains('|'.join(target), case=False).any(), axis=1)]
df_target.groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

#### Check environnemental

In [788]:
env = ['Albioma',
 'Alpha Metallurgical',
 'Ameren',
 'American Electric Power',
 'Arch Resources',
 'Athabasca',
 'BP p.l.c.',
 'British Petroleum',
 'Berry Corp',
 'Buzzi Unicem',
 'CF Industries',
 'Canadian Natural Resources',
 'Chevron',
 'Coronado Global Resources',
 'Denbury',
 'Electric Power Development',
 'Eni S.p.A.',
 'Eni SpA',
 'Entergy Corp',
 'Equinor',
 'Exxon',
 'HeidelbergCement',
 'Hokkaido Electric Power',
 'Hokuriku Electric Power',
 'Holcim',
 'New Hope',
 'OCI N.V.',
 'OMV AG',
 'Repsol',
 'Shell',
 'Stanmore',
 'Sumitomo Osaka Cement',
 'Taiheiyo Cement',
 'TerraCom',
 'AES Corp/The',
 "AES Corp",
 'The Scotts Miracle-Gro',
 'ScottsMiracle-Gro',
 'Southern Company',
 'Titan',
 'TransAlta',
 'TransAlta',
 'Vicat',
 'Vistra',
 'Soul Pattinson',
 'WHSP',
 'Yara',
 'eREX']

In [789]:
df_env = df[df.apply(lambda row: row.astype(str).str.contains('|'.join(env), case=False).any(), axis=1)]
df_env.groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

In [690]:
len(df_env)

43

In [691]:
df_env['Company Name'].value_counts().sort_values(ascending = False)

Eni SpA                           6
Holcim Helvetia Finance AG        5
Holcim Finance Luxembourg SA      4
Shell International Finance BV    4
Vistra Operations Co LLC          3
Holcim AG                         3
Canadian Natural Resources Ltd    2
Equinor ASA                       2
Yara International ASA            2
Entergy Corp                      2
Vistra Corp                       1
American Electric Power Co Inc    1
Shell PLC                         1
OMV AG                            1
Ameren Corp                       1
Chevron Corp                      1
Repsol SA                         1
Exxon Mobil Corp                  1
HeidelbergCement AG               1
CF Industries Holdings Inc        1
Name: Company Name, dtype: int64

#### Holcim

In [791]:
holcim = df[df.apply(lambda row: row.astype(str).str.contains('Holcim', case=False).any(), axis=1)]

In [792]:
holcim['Market Value CHF'].sum()

41558828

In [795]:
holcim['Date'].value_counts()

2022-06-30    8
2021-12-31    4
Name: Date, dtype: int64

#### 4 entreprises sans vergogne

In [768]:
select = ['repsol', 'yara', 'cf industries', 'lundin', 'Aker BP']

df[df.apply(lambda row: row.astype(str).str.contains('|'.join(select), case=False).any(), axis=1)]

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
2622,US55037AAA60,2021-12-31,AKERBP 2 07/15/26,LUNESS,Corporate Bonds,AKER BP ASA,Aker BP ASA,AKERBP 2 07/15/26,AKER BP ASA,Oil&Gas,Energy,Oil Comp-Explor&Prodtn,Netherlands,NL,NO,NO,Foreign Currency bonds,2 LUNDIN 26NTS-144A,USD 2'690'000,2303220,both
2933,US55037AAB44,2022-06-30,AKERBP 3.1 07/15/31,LUNESS,Corporate Bonds,AKER BP ASA,Aker BP ASA,AKERBP 3.1 07/15/31,AKER BP ASA,Oil&Gas,Energy,Oil Comp-Explor&Prodtn,Netherlands,NL,NO,NO,Foreign Currency bonds,3.1LUNDIN 31 NTS-,USD 3'830'000,3034263,both
3220,US984851AF24,2022-06-30,YARNO 4 ¾ 06/01/28,YARNO,Corporate Bonds,YARA INTERNATIONAL ASA,Yara International ASA,YARNO 4 ¾ 06/01/28,YARA INTL SA,Chemicals,Basic Materials,Agricultural Chemicals,Brazil,BR,BR,NO,Foreign Currency bonds,4.75 YR 28 NTS-144A,USD 2'160'000,1975649,both
3363,US00973RAE36,2022-06-30,AKERBP 3 01/15/25,AKERBP,Corporate Bonds,AKER BP ASA,Aker BP ASA,AKERBP 3 01/15/25,AKER BP ASA,Oil&Gas,Energy,Oil Comp-Explor&Prodtn,Norway,NO,NO,NO,Foreign Currency bonds,AKER 25 NTS-144A,USD 2'500'000,2316619,both
3812,NO0010345853,2022-06-30,AKER BP ASA,AKRBP,Common Stocks,Aker BP ASA,Aker BP ASA,AKER BP ASA,AKER BP ASA,Oil&Gas,Energy,Oil Comp-Explor&Prodtn,Norway,0,NO,NO,Equities,AKER BP RG,33'872,1122842,both
4040,US1252691001,2022-06-30,CF INDUSTRIES HOLDINGS INC,CF,Common Stocks,CF Industries Holdings Inc,CF Industries Holdings Inc,CF INDUSTRIES HOLDINGS INC,CF INDUSTRIES HO,Chemicals,Basic Materials,Agricultural Chemicals,United States,0,US,US,Equities,CF INDUSTRIES HL RG,10'561,866779,both
4577,CA5503721063,2022-06-30,LUNDIN MINING CORP,LUN,Common Stocks,Lundin Mining Corp,Lundin Mining Corp,LUNDIN MINING CORP,LUNDIN MINING CO,Mining,Basic Materials,Diversified Minerals,Chile,0,CL,CA,Equities,LUNDIN MINING RG,33'196,201036,both
4867,ES0173516115,2022-06-30,REPSOL SA,REP,Common Stocks,Repsol SA,Repsol SA,REPSOL SA,REPSOL SA,Oil&Gas,Energy,Oil Comp-Integrated,Spain,0,ES,ES,Equities,REPSOL BR,442'985,6229302,both
5236,NO0010208051,2022-06-30,YARA INTERNATIONAL ASA,YAR,Common Stocks,Yara International ASA,Yara International ASA,YARA INTERNATIONAL ASA,YARA INTL ASA,Chemicals,Basic Materials,Agricultural Chemicals,Brazil,0,BR,NO,Equities,YARA INTERNATION BR,17'879,713089,both


In [769]:
df[df.apply(lambda row: row.astype(str).str.contains('|'.join(select), case=False).any(), axis=1)].groupby('Company Name')['Market Value CHF'].sum()


Company Name
Aker BP ASA                   8776944
CF Industries Holdings Inc     866779
Lundin Mining Corp             201036
Repsol SA                     6229302
Yara International ASA        2688738
Name: Market Value CHF, dtype: int64

In [767]:
df[df.apply(lambda row: row.astype(str).str.contains('Aker BP', case=False).any(), axis=1)]['Market Value CHF'].sum()

8776944

#### Russie

In [770]:
russia = df[df.apply(lambda row: row.astype(str).str.contains('russi', case=False).any(), axis=1)]
len(russia)

1

In [771]:
russia

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
1919,XS2082345955,2021-12-31,CCHLN 0 ⅝ 11/21/29,CCHLN,Corporate Bonds,COCA-COLA HBC FINANCE BV,Coca-Cola HBC Finance BV,CCHLN 0 ⅝ 11/21/29,COCA-COLA HBC BV,Beverages,"Consumer, Non-cyclical",Beverages-Non-alcoholic,Russia,RU,IT,NL,Foreign Currency bonds,0.625 COCA CO29,EUR 2'500'000,2043259,both


#### Gold

In [776]:
df[df.apply(lambda row: row.astype(str).str.contains('Gold - LBMA Good Delivery Bars', case=False).any(), axis=1)]

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Nominal/Quantity,Market Value CHF,_merge
5513,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commodities,GOLD - LBMA GOOD DELIVERY BARS,NaN,1183595274,right_only


In [754]:
# Pour compléter ces montants, recherche dans le détail du fonds Pictet